In [1]:
import os
import sys
import statistics

aoc_year, aoc_day = os.getcwd().split(os.sep)[-2:]

# Download today puzzle & input
!aoc --version
!aoc download -i input.txt --overwrite -p README.md --year {aoc_year} --day {aoc_day}

aoc-cli 0.6.0
Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 11, 2022...
Saving puzzle description to "README.md"...
Saving puzzle input to "input.txt"...
Done!


\--- Day 11: Monkey in the Middle ---
----------

As you finally start making your way upriver, you realize your pack is much lighter than you remember. Just then, one of the items from your pack goes flying overhead. Monkeys are playing [Keep Away](https://en.wikipedia.org/wiki/Keep_away) with your missing things!

To get your stuff back, you need to be able to predict where the monkeys will throw your items. After some careful observation, you realize the monkeys operate based on *how worried you are about each item*.

...

In this example, the two most active monkeys inspected items 101 and 105 times. The level of *monkey business* in this situation can be found by multiplying these together: `*10605*`.

Figure out which monkeys to chase by counting how many items they inspect over 20 rounds. *What is the level of monkey business after 20 rounds of stuff-slinging simian shenanigans?*

In [2]:
from pprint import pprint

with open('input.txt', 'rt') as f:
    lines = [x.strip() for x in f.readlines()]

def monkey_parse(lines):
    monkeys = []
    while len(lines) > 0:
        monkey = {
            "name": lines.pop(0),
            "items": [int(_) for _ in lines.pop(0).replace("Starting items: ", "").split(", ")],
            "op": lines.pop(0).replace("Operation: ", ""),
            "test": int(lines.pop(0).split()[-1]),
            "true": int(lines.pop(0).split()[-1]),
            "false": int(lines.pop(0).split()[-1]),
        }
        monkeys.append(monkey)
        if len(lines) > 0: lines.pop(0)

    # Verify parse
    return monkeys


In [3]:
import math

def monkey_inspect(monkey, item, verbose=False):
    args = {"old": item}
    exec(monkey['op'], globals(), args)
    worry = math.floor(args['new'] / 3)
    if verbose: print(f"{monkey['name']} {monkey['op']} {item}->{args['new']}->{worry} %% {monkey['test']}")
    if worry % monkey['test'] == 0:
        return monkey['true'], worry
    else:
        return monkey['false'], worry

#monkey_inspect(monkeys[0], 62, True)

In [4]:
monkeys = monkey_parse(lines.copy())

count = [0 for _ in range(len(monkeys))]
for round in range(20):
    for i, monkey in enumerate(monkeys):
        while len(monkeys[i]['items']) > 0:
            item = monkeys[i]['items'].pop(0)
            dest, worry = monkey_inspect(monkeys[i], item)
            count[i] += 1
            #print(f"{round+1}| {monkey['name']} {item} -> {dest} ({worry})")
            monkeys[dest]['items'].append(worry)

count

[291, 296, 17, 297, 306, 7, 6, 115]

In [5]:
answer1 = sorted(count)[-1] * sorted(count)[-2]
print("answer1:", answer1)

answer1: 90882


----

In [6]:
# Download part 2
!aoc download --description-only --overwrite --puzzle-file README.md --year {aoc_year} --day {aoc_day}

Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 11, 2022...
Saving puzzle description to "README.md"...
Done!


\--- Part Two ---
----------

You're worried you might not ever get your items back. So worried, in fact, that your relief that a monkey's inspection didn't damage an item *no longer causes your worry level to be divided by three*.

Unfortunately, that relief was all that was keeping your worry levels from reaching *ridiculous levels*. You'll need to *find another way to keep your worry levels manageable*.

...

Worry levels are no longer divided by three after each item is inspected; you'll need to find another way to keep your worry levels manageable. Starting again from the initial state in your puzzle input, *what is the level of monkey business after 10000 rounds?*


In [7]:
def monkey_inspect_prime(monkey, item, modulo, verbose=False):
    args = {"old": item}
    exec(monkey['op'], globals(), args)
    worry = args['new'] #math.floor(args['new'] / 3)
    if verbose: print(f"{monkey['name']} {monkey['op']} {item}->{args['new']}->{worry} %% {monkey['test']}")
    if worry % monkey['test'] == 0:
        return monkey['true'], worry % modulo
    else:
        return monkey['false'], worry % modulo


In [8]:
# What number is divisible by many? The product! :headthump:
monkeys = monkey_parse(lines.copy())
modulo = math.prod([m['test'] for m in monkeys])
count = [0 for _ in range(len(monkeys))]

for round in range(10_000):
    for i, monkey in enumerate(monkeys):
        while len(monkeys[i]['items']) > 0:
            item = monkeys[i]['items'].pop(0)
            dest, worry = monkey_inspect_prime(monkeys[i], item, modulo)
            count[i] += 1
            #print(f"{round+1}| {monkey['name']} {item} -> {dest} ({worry})")
            monkeys[dest]['items'].append(worry)

count

[170248, 174243, 3549, 161508, 177299, 16284, 28995, 28223]

In [9]:
answer2 = sorted(count)[-1] * sorted(count)[-2]
print("answer 2:", answer2)

answer 2: 30893109657
